In [15]:
'''
Line extraction
This class extract which smrt line sentence belongs to 
'''
import re
from enum import Enum

class StationLine(Enum):
    red = 1
    green = 2
    circle = 3
    purple = 4
    blue = 5
    brown = 6
    none = 99

class ExtractStationLine:

    redLineList = ["[nsl]","[nsl]:", "ns", "nsl", "north south", "north south line", "nsline", "ns line"]
    greenLineList = ["[ewl]","[ewl]:","ew", "ewl", "east west", "east west line", "ewline", "ew line"]
    circleLineList = ["[ccl]","[ccl]:","cc", "ccl", "circle line", "circleline", "cc line"]
    purpleLineList = ["[nel]","[nel]:","ne", "nel", "north east", "north east line", "neline", "ne line"]
    blueLineList = ["[dtl]","[dtl]:","dt", "dtl", "down town", "down town line", "dtline", "dt line"]
    brownLineList = ["[tecl]","[tecl]:","tec", "tecl", "thomson-east Coast", "thomson-east Coast line", "tecline", "tec line"]
    
    def checkStationLine(self, jsonData):
        if self.containKey(StationLine.red, jsonData):
            return StationLine.red
        elif self.containKey(StationLine.green, jsonData):
            return StationLine.green
        elif self.containKey(StationLine.circle, jsonData):
            return StationLine.circle
        elif self.containKey(StationLine.purple, jsonData):
            return StationLine.purple
        elif self.containKey(StationLine.blue, jsonData):
            return StationLine.blue
        elif self.containKey(StationLine.brown, jsonData):
            return StationLine.brown
        else:
            return StationLine.none
            
    def containKey(self, stationType, jsonData):
        if stationType is StationLine.red:
            lineList = self.redLineList
        elif stationType is StationLine.green:
            lineList = self.greenLineList
        elif stationType is StationLine.circle:
            lineList = self.circleLineList
        elif stationType is StationLine.purple:
            lineList = self.purpleLineList
        elif stationType is StationLine.blue:
            lineList = self.blueLineList
        elif stationType is StationLine.brown:
            lineList = self.brownLineList
#         print(lineList)
        
        isContainKey = False
        for key in lineList:    
            result = self.findWholeWord(key)(jsonData)
            if result is not None:        
                isContainKey = True
                break
        return isContainKey

    def findWholeWord(self, w):
        return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

extStLine = ExtractStationLine()
extStLine.checkStationLine("This sentence contain key words for blah blah some rubbish words train station line")

<StationLine.none: 99>